In [3]:
import  numpy as  np
import  pandas as  pd
import re
import itertools
import copy
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import collections
from collections import defaultdict
mystem = Mystem()
morph = MorphAnalyzer()

## the Levenshtein distance

In [4]:
data = pd.read_csv('fon_table.csv/data.csv')
data = data.set_index('name')
data = data.transpose()
data = data.fillna('None')
data = pd.DataFrame(data)
data.head()

name,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
son,+,+,+,+,+,+,+,+,+,+,...,+,+,-,-,-,-,-,-,-,+
voc,+,+,+,+,+,+,+,+,+,+,...,+,-,-,-,-,-,-,-,-,-
cons,-,-,-,-,-,-,-,-,-,-,...,+,+,+,+,+,+,+,+,+,+
cor,None,None,None,None,None,None,None,None,None,None,...,+,-,-,-,-,-,-,-,-,-
ant,None,None,None,None,None,None,None,None,None,None,...,-,-,-,-,-,-,-,-,-,-


In [ ]:
data.to_csv('data.csv')

In [5]:
dist_matrix = pd.DataFrame(np.zeros((54, 54)))
column_names = {}

for i in range(0, 54):
    column_names[i] = data.columns.tolist()[i]
    
dist_matrix = dist_matrix.rename(index=str, columns=column_names)
dist_matrix['index'] = data.columns.tolist()
dist_matrix = dist_matrix.set_index('index')

Заполняю клеточки в таблице результатами формулы

1 - S_rows/ C_rows + Unc_rows*2

In [6]:
comobinations = itertools.combinations_with_replacement(data.columns, 2) # - все возможные комбинации

In [7]:
for i in comobinations:

    a1 = pd.get_dummies(data[i[0]]).drop(['None'], axis=1)
    a = a1.loc[~(a1==0).all(axis=1)]
    b1 = pd.get_dummies(data[i[1]]).drop(['None'], axis=1)
    b = b1.loc[~(b1==0).all(axis=1)]
    
    dif_a = a.index.difference(b.index)
    similar = a.index.difference(dif_a)
    
    d = a.loc[similar] + b.loc[similar]
    s_rows = 0
    counts = d['+'].value_counts()

    if 0 in counts:
        s_rows += counts[0]
    if 2 in counts:
        s_rows += counts[2]
    
                         # common                uncommon
    result = 1 - s_rows/(len(similar) + 2*(a.shape[0] + b.shape[0] - 2*len(similar)))

    dist_matrix[i[0]][i[1]] = result
    dist_matrix[i[1]][i[0]] = result

In [8]:
dist_matrix.head()

,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
index,,,,,,,,,,,,,,,,,,,,,
и,0.000,0.125,0.125,0.250,0.250,0.375,0.250,0.125,0.375,0.375,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
ы,0.125,0.000,0.250,0.125,0.375,0.250,0.125,0.250,0.250,0.250,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
е,0.125,0.250,0.000,0.125,0.125,0.250,0.375,0.250,0.250,0.250,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
э,0.250,0.125,0.125,0.000,0.250,0.125,0.250,0.375,0.125,0.125,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
ä,0.250,0.375,0.125,0.250,0.000,0.125,0.500,0.375,0.375,0.375,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333


In [ ]:
dist_matrix.to_csv('distance_matrix.csv')

In [ ]:
f = open('ru_st_normalized.csv', 'a', encoding='utf-8')
for index, row in st_words.iterrows():
    f.write(str(index) + ';' + str(row['stressed_s']) + '\n')
f.close()

In [ ]:
f = open('distance_matrix.csv', 'w', encoding='uft-8')
for index, row in dist_matrix.iterrows():
    f.write(str(index) + ';' + str(row['stressed_s']) + '\n')

## Расстояние Левенштейна

In [9]:
def lev_distance(a, b):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
  
    return dis[len(a), len(b)]

## Фонетические изменения

ГЛАСНЫЕ

In [10]:
vows = pd.read_csv('vowels.csv', sep=';')
vows = vows.set_index('Unnamed: 0')
vows = vows.reindex(vows.index.rename('name'))
vows

,#,v2_hard,v2_soft,v1_hard,v1_sh,v1_soft,V,vn_hard,vn_soft
name,,,,,,,,,
а,а,ъ,ь,а,a,и,а,ъ,ь
я,а,ъ,ь,а,a,и,а,ъ,ь
о,а,ъ,ь,а,ы,и,о,ъ,ь
ё,а,ъ,ь,а,ы,и,о,ъ,ь
у,у,у,у,у,у,у,у,у,у
ю,у,у,у,у,у,у,у,у,у
е,и,ъ,ь,ы,ы,и,е,ъ,ь
э,и,ъ,ь,ы,ы,,э,ъ,ь
и,и,ъ,ь,ы,ы,и,и,ъ,ь


In [ ]:
vows.to_csv('vows.csv')

СОГЛАСНЫЕ

In [11]:
cons = pd.read_csv('cons.csv', sep=';')
cons = cons.set_index('Unnamed: 0')
cons = cons.reindex(cons.index.rename('name'))
cons

,soft,hard,voiced,no_voice
name,,,,
б,б’,б,б,п
в,в’,в,в,ф
г,г’,г,г,к
д,д’,д,д,т
ж,ж,ж,ж,ш
з,з’,з,з,с
й,j,j,j,j
к,к’,к,г,к
л,л’,л,л,л


In [ ]:
cons.to_csv('cons_.csv')

Таблица, которая помогает учесть ассимиляцию

In [12]:
aff = pd.read_csv('aff.csv', sep=';')
aff = aff.set_index('Unnamed: 0')
aff = aff.reindex(aff.index.rename('name'))
aff

,с,ш,щ,ч,ж
name,,,,,
с,-,ш,ш’,ш’,ж
сс,-,-,-,ш’,-
ст,-,-,-,ш’,-
з,-,ш,ш’,ш’,ж’
зд,-,-,ш’,ш’,-
т,ц,ч’,-,ч’,-
ть,ц,-,-,-,-
д,ц,ч’,-,-,-
ж,-,-,-,ш’,ж’


In [ ]:
aff.to_csv('aff_.csv')

### ТАБЛИЦА УДАРЕНИЙ

In [13]:
st_words = pd.read_csv('ru_reversed.csv', sep='\t')
new = st_words['word'].str.replace(r'[^\w\s]', '')
new = new.str.lower()
st_words['word'] =  new
st_words = st_words.drop_duplicates()
st_words = st_words.set_index('word')
st_words.head()

,stressed_s
word,
а,1
аа,2
ааа,3
аааа,4
ааааа,5


In [ ]:
st_words.to_csv('stress_data.csv')

### ТРАНСКРИПЦИЯ

Кол-во гласных в слова

In [14]:
def num_of_vowls(word):
    num = len(re.findall('(а|е|ё|о|и|я|ю|у|ы|э)', word))   
    
    if num:
        return num  
    else:
        return 1

In [14]:
num_of_vowls('мама')

2

In [15]:
num_of_vowls('vfa')

1

Функция, которая ищет заданное слово в словаре с ударениями. Возращает номер гласной, на которой стоит ударение. Чтобы не использовать дополнительную память, сделала из него генератор. Если слова в словаре нет, то предлагаются все возможные варинты

In [15]:
def stressed(word, st_words):
    
    try:
        a = st_words.loc[word]['stressed_s'].tolist()
   
        if isinstance(a, list):
            yield from a
        else:
            yield from [a]
    
    except KeyError:
        yield from range(1, num_of_vowls(word) + 1)

In [16]:
[i for i in stressed('замок', st_words)]

[2, 1]

In [17]:
for i in stressed('замок', st_words):
    print(i)

2
1


Изменения гласных

In [18]:
def due_to_vow_table(vow_n, i, letter, index, length,  letters, soft, j_):

    ans = letter

    if vow_n == i: # ударный 

        if letter in ('е', 'ё', 'и', 'ю', 'я', 'э') and index + 1 <= length-1 and letters[index+1] not in vows.index:
            if letter != 'э' and letters[index+1] not in ('ш', 'ж', 'ц'):
                soft = True
            elif letter in ('э', 'е') and letters[index+1] in ('ш', 'ж', 'ц'):
                letter = 'э'
                
        ans, j_ = j(letter, index, length, vows, letters, 'V', j_)
        
    elif index == length-1:  # начало
        ans, j_ = j(letter, index, length, vows, letters, '#', j_)
        
        
    elif vow_n == i + 1: # первый предударный

        if letters[index+1] in ('ц', 'ж', 'ш'):
            ans = vows.loc[letter]['v1_sh']
        
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans = vows.loc[letter]['v1_soft']
            soft = True
        else:
            ans = vows.loc[letter]['v1_hard']
            
            
    elif vow_n >= i + 2: # второй предударный
        
        if letters[index+1] in ('ц', 'ж', 'ш'):
            ans = vows.loc[letter]['v2_hard']
        
        elif letters[index+1] in vows.index:
            if letter in ('е', 'ё', 'и', 'ю', 'я'):
                ans = vows.loc[letter]['v1_soft']      
            else:
                ans = vows.loc[letter]['v1_hard']
                
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans = vows.loc[letter]['v2_soft']
            soft = True
        else:
            ans = vows.loc[letter]['v2_hard']
            
            
    elif vow_n < i: # заударные 
        
        if index + 1 <= length-1 and letters[index+1] in ('ц', 'ж', 'ш'):
   
            ans, j_ = j(letter, index, length, vows, letters, 'vn_hard', j_)
            #vows.loc[letter]['vn_hard']
            #j(letter, index, length, vows, letters, 'vn_hard')
   
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            
            if vow_n == i - 1 and index + 1 <= length-1 and letters[index+1] in vows.index:
                ans, j_ = j(letter, index, length, vows, letters, 'vn_soft', j_)
            else:
                ans = vows.loc[letter]['vn_soft']
            # j(letter, index, length, vows, letters, 'vn_soft')
          
            soft = True
        else:
            if vow_n == i - 1 and index + 1 <= length-1 and letters[index+1] in vows.index:
                ans, j_ = j(letter, index, length, vows, letters, 'vn_hard', j_)
            else:
                ans = vows.loc[letter]['vn_hard']
            #j(letter, index, length, vows, letters, 'vn_hard')
           
    return ans, soft, j_

In [23]:
def j(letter, index, length, vows, letters, position, j_):

    ans = vows.loc[letter][position]

    if letter in ('ю', 'е', 'ё', 'я'):
        
        if index == length - 1:
            j_ = True
           # ans = 'j' + ans
    
        elif index + 1 <= length-1 and letters[index+1] in vows.index:
            j_ = True
            if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']
            #ans = 'j' + ans
        
        elif index + 1 <= length-1 and letters[index+1] in ('ь', 'ъ'):
            j_ = True
            if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']
            #ans = 'j' + ans
    
    elif letter in ('и', 'о') and index + 1 <= length-1 and letters[index+1] == 'ь':
        j_ = True
        if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']
        #ans = 'j' + ans
    
    return ans, j_

In [24]:
def vows_after_aff(ans2, index, answer):
    
    if data[ans2]['back'] == '+' and index > 0:
        if answer[-1] == 'е':
            answer[-1] = 'э'
        if answer[-1] == 'и':
            answer[-1] = 'ы'
        if answer[-1] == 'ь':
            answer[-1] = 'ъ'

    elif data[ans2]['back'] == '-' and index > 0:
        if answer[-1] == 'э':
            answer[-1] = 'е'
        if answer[-1] == 'ы':
            answer[-1] = 'и'
        if answer[-1] == 'ъ':
            answer[-1] = 'ь'
 

In [25]:
def cons_tranformer(letter, no_voice, soft, voice, index, length, letters, gap, answer, vcd=False):

    if gap:  # пропуск в случае ассимиляции
        return '', no_voice, soft, voice, gap

    if letter in ('ч', 'ш', 'щ', 'ж'):
        ans = cons.loc[letter]['hard']
        soft = False
        if letter in ('ч', 'щ'):
            no_voice = False
            voice = False
    else:
        ans = letter

    if no_voice and ans:  # оглушение
        ans = cons.loc[ans]['no_voice']

    elif voice:  # озвончение
        ans = cons.loc[ans]['voiced']

    if vcd is True:
        if index == 0 or index-1 == 0 and letters[index-1] in ('ь', 'ъ'):
            ans = cons.loc[ans]['voiced']

    if letter in aff.columns and index + 1 <= length-1 and (letters[index+1] in cons.index or letters[index+1] == 'ь'):  # особая ассимиляция

        if index + 2 <= length-1:

            var = letters[index+2] + letters[index+1]

            if var in aff.index:
 
                ans2 = aff.loc[var][letter]

                if ans2 != '-':
                    gap = True
                    vows_after_aff(ans2, index, answer)
                    return ans2, no_voice, soft, voice, gap

        if letters[index+1] in aff.index:

            ans2 = aff.loc[letters[index+1]][letter]

            if ans2 != '-':
                gap = True
                vows_after_aff(ans2, index, answer)
                return ans2, no_voice, soft, voice, gap

    if not vcd:
        if index == 0 or index-1 == 0 and letters[index-1] in ('ь', 'ъ'):  # конец слова
            no_voice = True
            if data[ans]['vcd'] == '+':
                ans = cons.loc[ans]['no_voice']

    elif ans in data.columns and data[ans]['son'] == '-' and ans not in ('в', "в’") and data[ans]['vcd'] == '+':  # озвончение слудующих
        voice = True

    elif ans in data.columns and data[ans]['vcd'] == '-' and index + 1 <= length-1:  # оглушение следующих
        if letters[index+1] not in ('ь', 'ъ') and letters[index+1] in data.columns and data[letters[index+1]]['son'] == '-':
            no_voice = True
        elif index + 2 <= length-1 and letters[index+2] in data.columns and data[letters[index+2]]['son'] == '-':
            no_voice = True

    if index-1 >= 0:

        if letters[index-1] == 'к' and letter == 'г':
            ans = 'х'
        
        elif len(answer) > index-1 and answer[index-1] == 'т' and ans == "ч’":
            ans = 'ш'

        elif len(answer) > index-1 and answer[index-1] in (ans, ans + "’"):
            return '', no_voice, soft, voice, gap

    if soft:  # смягчение
        ans = cons.loc[ans]['soft']

    ## if ans not in ('ṷ', 'j', "ш’", "ч’") and index > 0 and answer[-1] in data.columns and data[answer[-1]]['back'] == '-': # смягчение после:
    ##     if data[answer[-1]]['cor'] in ('-', '+') and data[answer[-1]]['ant'] == '+': # мягких губных или зубных
    ##        ans = cons.loc[ans]['soft']

    soft = False
    return ans, no_voice, soft, voice, gap

In [26]:
def transcription(word, stress=False, next_word=False, stop_word=False, separate=True, stop=False, vcd=False):
    
    """
    Фунция, которая определяет фонетическую транскриацию для слова с заданным ударением,
    расчитываемым с конца слова. 
    
    Функция принимает 3 аргемента: 
    i - type int, номер гласного, на который падает ударение (отсчет производится с конца слова).
    word - type str, слово, для которого должен произвожится разбор. 
    
    >>> transcription(2, 'мама')
    ['м', 'а', 'м', 'ъ']
    
    >>> transcription(1, 'мама')
    ['м', 'а', 'м', 'а']
    
    
    Функция работает только с кириллическими символами. Если в слове содержатся не кириллические символы, вызывается ошибка.
    
    >>> transcription(2, 'papa')
    Traceback (most recent call last):
        ...
    ValueError: No vowles or not Cyrillic elements
    
    
    Если пользователь указывает номер гласной меньший кол-ва гласных, вызывается ошибка.
    
    >>> transcription(3, 'мама')
    Traceback (most recent call last):
        ...
    ValueError: There are only 2 vowel(s)
    """
    
    if not isinstance(word, str):
        raise ValueError('Wrong data type')
        
    if not isinstance(stress, int):
        raise ValueError('Wrong data type')
        
    if ' ' in word:
        raise ValueError('Enter a word, not a phrase')
        
    nums = num_of_vowls(word)

    if stress:
        if nums < stress or stress <= 0:
            raise ValueError('There are only {} vowel(s)'.format(str(num_of_vowls(word))))
        
    if not stress:
        stress = nums
    
    if stop == True:
        stress = -100000
    
    letters = list(word)[::-1]
    answer = []
    length = len(word)
    vow_n = 0
    soft = False
    voice = False
    no_voice = False
    gap = False
    j_ = False
    special = False
        
    for index, letter in enumerate(letters):

        letter = letter.lower()
    
        if letter in vows.index:
        #if vows.loc[letter]:
        
            voice = False
            no_voice = False
            gap = False
            soft = False
            vow_n += 1
        
            ans, soft, j_ = due_to_vow_table(vow_n, stress, letter, index, length, letters, soft, j_)
            
            if j_ is True and index == 0 and vow_n != stress:
                special = True
    
        elif letter == 'ь' or letter == 'ъ': 
            #ans = ''
        
            if letter == 'ь':
            
                if index - 1 >= 0 and letters[index-1] in aff.columns:
        
                    if index + 1 <= length-1 and letters[index+1] + letter in aff.index: 
                        gap = True
                       # answer.pop(-1)#answer[-1] = ''
                       # ans = aff.loc[letters[index+1] + letter][letters[index-1]]
                       # gap = True
            
                else:
                    soft = True
                
            if letter == 'ъ':
                soft = False
            continue
    
        elif letter in cons.index:
        
            if letter == 'й':
                ans = 'ṷ'
            
            else:
                ans, no_voice, soft, voice, gap = cons_tranformer(letter, no_voice, soft, voice, index, length, letters, gap, answer, vcd=vcd)
        
                if ans == '':
                    continue
        else:

            raise ValueError('Not Cyrillic')
        
        answer.append(ans)

        if j_:
            if special:
                answer.append('ṷ')
                special = False
            else:
                answer.append('j')
            j_ = False
            

    if separate == False:
        return ''.join(answer[::-1])
    return answer[::-1]

In [27]:
transcription('потчевать', stress=3, separate=False)

'поч’ьвът’'

In [28]:
transcription('рассвет', stress=1, separate=False)

'расв’ет'

In [29]:
stop_words = ['а',  'без',  'близ',  'в',  'вне',  'во',  'вокруг',  'вслед',  'для',  'до',  'за',  'и',  'из', 
              'изза',  'изо',  'изпод',  'к',  'ко',  'меж',  'между',  'мимо',  'на',  'над',  'о',  'об',  'обо', 
              'около',  'от',  'ото',  'перед',  'передо',  'по', 'поверх',  'под',  'подо',  'понад',  'после', 
              'пред',  'при',  'про',  'ради',  'с',  'сверх',  'сверху']

In [30]:
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text).replace('\n', '')
    text = re.sub(r'[\s]{2,}',' ',text)
    return text.split(' ')

In [31]:
def phrase_transformer(text, separate=True):
    
    def combine(terms, accum):

        last = (len(terms) == 1)
        n = len(terms[0])

        for i in range(n):
            item = accum + [terms[0][i]]

            if last:
                combinations.append(item)
            else:
                combine(terms[1:], item)
                
    if not isinstance(text, str):
        raise ValueError('Wrong data type')

    num_words = 0
    words = tokenize(text)
    answer = []
    combinations = []
    length = len(words)

    for index, word in enumerate(words):

        stop = False
        vcd = False  # озвончение, если это фраза 
   
        if word in stop_words and length > 1:
            stop = True
            if index+1 <= len(words)-1 and words[index+1][0] in data.columns and data[words[index+1][0]]['vcd'] == '+':
                vcd = True
        
        answer.append([])
        
        for stress in stressed(word, st_words):
            num_words+=1
            if separate == False:
             
                answer[-1].append(''.join(transcription(word, stop=stop, vcd=vcd, stress=stress)))

            else:
                answer[-1].append(transcription(word, stop=stop, vcd=vcd, stress=stress))

    combine(answer, [])
    return combinations

In [33]:
a = phrase_transformer('замок')
for i in a:
    print(i)


[['з', 'а', 'м', 'ъ', 'к']]
[['з', 'а', 'м', 'о', 'к']]


In [34]:
phrase_transformer('в комнате', separate=False)#[0]

[['ф', 'комнът’ь']]

In [35]:
def phonetic_distance(word1, word2=False, transcription=False):
    
    word1 = phrase_transformer(word1)
        
    if len(word1[0]) > 1:
        raise ValueError('Enter values must be words, not phrases')
            
    word2 = phrase_transformer(word2)
        
    if len(word1[0]) > 1:
        raise ValueError('Enter values must be words, not phrases')

    answer = []
        
    for w1 in word1:
        for w2 in word2:

            if transcription:
                answer.append([''.join(w1[0]), ''.join(w2[0]), lev_distance(w1[0], w2[0])])
            else:
                answer.append(lev_distance(w1[0], w2[0]))
        
    return answer

In [36]:
transcription('под', stop_word=True)

['п', 'о', 'т']

In [37]:
phonetic_distance('замок', 'замок')

[0.0, 0.25, 0.25, 0.0]

In [ ]:
господи
ага
бога
бугалтер